### Testing with pre-trained EGNN

In [23]:
import torch
import EGNN
import dataset_ani1

In [24]:
model = EGNN.EGNN(
    hidden_channels=256, in_edge_nf=0,
    n_layers=5, residual=True, 
    attention=True, normalize=True, 
    tanh=False, max_atom_type=28, 
    cutoff=5.0, max_num_neighbors=32
)

pretrained_dict = torch.load('model.pth', map_location=torch.device('cpu'))

model.load_state_dict(pretrained_dict)

<All keys matched successfully>

In [25]:
from torch.nn.functional import mse_loss
def rmse_loss(input, target):
    """
    Calculate the Root Mean Squared Error between the predicted and target tensors.
    """
    return torch.sqrt(mse_loss(input, target))

In [28]:
from torch.nn import MSELoss

data_dir = '../Data'  # Replace with the actual path to your dataset
batch_size = 32
num_workers = 4
valid_size = 0.1
test_size = 0.1
seed = 42  # For reproducibility

# Instantiate the wrapper
dataset_wrapper = dataset_ani1.ANI1Wrapper(
    batch_size=batch_size,
    num_workers=num_workers,
    valid_size=valid_size,
    test_size=test_size,
    data_dir=data_dir,
    seed=seed
)
# Obtain the data loaders
train_loader, valid_loader, test_loader = dataset_wrapper.get_data_loaders()

# Now you can use train_loader, valid_loader, and test_loader in your training and evaluation loops.
model.eval()
loss_function = MSELoss()

accumulated_loss = 0.0
total_samples = 0

for batch in valid_loader:
    # Extract data and move tensors to the same device as the model
    # Assuming your model and data are on the same device (CPU or CUDA)
    # If your batch contains more than these two elements, unpack accordingly
    inputs, pos, batch_index = batch.x, batch.pos, batch.batch
    targets = batch.y
    # Forward pass: compute predicted outputs by passing inputs to the model
    outputs = model(inputs, pos, batch_index)

    predictions = outputs if isinstance(outputs, torch.Tensor) else outputs[0]
    # Compute the loss
    loss = rmse_loss(predictions, targets)
    # Update the total loss and the number of samples
    accumulated_loss += loss.item() * targets.size(0)  # Multiply by batch size
    total_samples += targets.size(0)

print(total_samples)
# Calculate the average loss over all of the batches
average_loss = accumulated_loss / total_samples
average_loss

reading: ani1_gdb10_ts.h5
# molecules: 138
# train conformations: 8416
# valid conformations: 966
# test conformations: 966
960


2111.1462646484374

: 